In [4]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import glob
import matplotlib.pyplot as plt
from scipy.spatial import distance
from math import inf
import face_recognition as fr

In [5]:
import time
import os

In [8]:
# technique from the following sources
# https://sefiks.com/2020/08/25/deep-face-detection-with-opencv-in-python/
# https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/
#load the face detector for opencv
detector = cv2.dnn.readNetFromCaffe("deploy.prototxt",
                                    "res10_300x300_ssd_iter_140000.caffemodel")

# make a dictionary containing the reference images for the candidates
reference = {}
numbers = list(range(0,10))
for image in glob.glob("reference_images/*.jpg"):
    candidate = image.replace("reference_images/", '')
    candidate = candidate.replace(".jpg", '')
    for number in numbers:
        candidate = candidate.replace(str(number), '')
    real_face = cv2.imread(image)
    reference[candidate] = real_face   

start = time.time()    
vid_number = 1
# iterate through the videos
for video in glob.glob("./video_2020/VideoClips/*.mp4"):
    
    # extract the video path
    video_path = video.replace("./video_2020/VideoClips/", '')
    video_path = video_path.replace(".mp4", '')
    
#     if any(video_path in directory for directory in existing):
#         continue
    
    # make two new directories to store the face image and the face image in gray
    os.mkdir(f"./299_verified_2020/{video_path}")
    os.mkdir(f"./299_verified_2020_gray/{video_path}")
    
    # extract the candidate name
    candidate = video.replace("./video_2020/VideoClips/", '')
    candidate = candidate.replace(".mp4", '')
    candidate = candidate.replace("_", "")
    
    for number in numbers:
        candidate = candidate.replace(str(number), '')
    
    # read the video
    video = cv2.VideoCapture(video)

    # count the number of frames read
    count = 0
    success = True

    # use a loop to get each frame from the video
    while success:

        # get each frame, detect whether a frame was successfully extracted
        success, image = video.read()

        if success == False:
            break
            
        base_img = image.copy()

        original_size = base_img.shape

        target_size = (300, 300)
        image = cv2.resize(image, target_size)

        aspect_ratio_x = (original_size[1] / target_size[1])
        aspect_ratio_y = (original_size[0] / target_size[0])

        imageBlob = cv2.dnn.blobFromImage(image = image)
        detector.setInput(imageBlob)
        detections = detector.forward()

        # get a dataframe of all the faces detected in teh image
        column_labels = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"]
        detections_df = pd.DataFrame(detections[0][0], columns = column_labels)
        detections_df = detections_df[detections_df['is_face'] == 1]
        detections_df = detections_df[detections_df['confidence']>=0.60]
        detections_df['left'] = (detections_df['left'] * 300).astype(int)
        detections_df['bottom'] = (detections_df['bottom'] * 300).astype(int)
        detections_df['right'] = (detections_df['right'] * 300).astype(int)
        detections_df['top'] = (detections_df['top'] * 300).astype(int)
        
        if len(detections_df) == 0:
            continue

        shortest = inf
        closest = 0

        # go through each detected face and compare to the reference image
        # store the most similar face
        for i, instance in detections_df.iterrows():
            confidence_score = str(round(100*instance["confidence"], 2))+"%"
            left = int(instance["left"])
            right = int(instance["right"])
            bottom = int(instance["bottom"])
            top = int(instance["top"])

            detected_face = base_img[int(top*aspect_ratio_y):int(bottom*aspect_ratio_y), int(left*aspect_ratio_x):int(right*aspect_ratio_x)]

            try:
                final_image = cv2.resize(detected_face, (224, 224))
            except:
                continue
                
            final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2GRAY)

            reference_image = reference[candidate]

            reference_image = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)

            cos_dist = distance.cosine(final_image.ravel(), reference_image.ravel())

            if cos_dist < shortest:
                shortest = cos_dist
                closest = i
            
        # save the confirmed face to the relevant directory
        confirmed_face = detections_df.loc[i]

        left = int(confirmed_face["left"])
        right = int(instance["right"])
        bottom = int(confirmed_face["bottom"])
        top = int(confirmed_face["top"])

        confirmed_face = base_img[int(top*aspect_ratio_y):int(bottom*aspect_ratio_y), int(left*aspect_ratio_x):int(right*aspect_ratio_x)]

        confirmed_face = cv2.resize(confirmed_face, (299, 299))

        cv2.imwrite(f"299_verified_2020/{video_path}/frame{count}.jpg", confirmed_face)

        count += 1
        
    current_time = time.time()
    if vid_number % 10 == 0:
        duration = current_time - start
        print(f"{vid_number} videos processed")
        print(f"Time elapsed: {duration}")
        time_per_video = duration/vid_number
        remaining_videos = 1506 - vid_number
        estimated_time_remaining = time_per_video*remaining_videos
        print(f"Estimated time remaining: {estimated_time_remaining}")

    vid_number += 1

10 videos processed
Time elapsed: 27.73526906967163
Estimated time remaining: 4149.196252822876
20 videos processed
Time elapsed: 53.19948482513428
Estimated time remaining: 3952.721722507477
30 videos processed
Time elapsed: 88.75356364250183
Estimated time remaining: 4366.67533121109
40 videos processed
Time elapsed: 112.9350483417511
Estimated time remaining: 4139.069521725178
50 videos processed
Time elapsed: 147.7690281867981
Estimated time remaining: 4303.03410079956
60 videos processed
Time elapsed: 188.28458642959595
Estimated time remaining: 4537.658532953263
70 videos processed
Time elapsed: 201.52779459953308
Estimated time remaining: 4134.198757784707
80 videos processed
Time elapsed: 231.3229479789734
Estimated time remaining: 4123.331547725201
90 videos processed
Time elapsed: 255.9573531150818
Estimated time remaining: 4027.0623556772866
100 videos processed
Time elapsed: 287.94150495529175
Estimated time remaining: 4048.457559671402
110 videos processed
Time elapsed: 32